### Importacion de Librerias

In [86]:
import simpy
import pandas as pd
import numpy as np
import random

### Definicion de clases, recursos y containers

In [87]:
class Piso:
    def __init__(self, env, id, num_parqueos):
        self.env = env
        self.id = id
        self.num_parqueos = simpy.Container(
            env,
            init=num_parqueos,
            capacity=num_parqueos
        )

In [88]:
class Estacionamiento:
    def __init__(self, env, parquimetros, pisos):
        self.env = env
        self.pisos = pisos
        self.parquimetros = simpy.Resource(env, capacity=parquimetros)

In [ ]:
class Carro:
    def __init__(self,env, id, piso, estacionamiento):
        self.env = env
        self.id = id
        self.piso: Piso = piso
        self.estacionamiento: Estacionamiento = estacionamiento
        self.env.process(self.start())
        self.encontro_parqueo = False
        self.eventos = pd.DataFrame()

    def start(self):

        piso_elegido = self.piso
        piso_asignado = None

        print(f"{self.env.now}: el carro con id {self.id} llego y quiere estacionarse en el piso {piso_elegido.id}")
        self.eventos = pd.concat( [self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": ["LLEGO"],
            "time": [self.env.now]
        })] )

        parquimetro = self.estacionamiento.parquimetros

        with parquimetro.request() as request:
            yield request
            yield self.env.timeout(random.randint(0 , 1))
            print(f"{self.env.now}: El carro con id {self.id} esta entrando al estacionamiento")
            self.eventos = pd.concat( [self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["ENTRO"],
                "time": [self.env.now]
            })] )

            #Mira si hay parqueos disponibles en el piso preferido por el carro
            if piso_elegido.num_parqueos.level > 0:
                yield self.env.timeout(random.randint(1 , 3))
                yield self.piso.num_parqueos.get(1)                
                piso_asignado = piso_elegido
                self.encontro_parqueo = True
                print(f"{self.env.now}: El carro {self.id} se estaciono en el piso que queria ({piso_asignado.id})")
                self.eventos = pd.concat( [self.eventos, pd.DataFrame({
                    "id": [self.id],
                    "evento": ["ESTACIONO"],
                    "time": [self.env.now]
                })] )
            else:
                #Busca parqueo en otro piso
                for opcion_piso in self.estacionamiento.pisos:
                    if opcion_piso.num_parqueos.level > 0:
                        yield opcion_piso.num_parqueos.get(1)
                        piso_asignado = opcion_piso
                        print(f"{self.env.now}: El carro {self.id} no encontro espacio en el piso {piso_elegido.id} pero se estaciono en el piso {opcion_piso.id}")
                        self.eventos = pd.concat( [self.eventos, pd.DataFrame({
                            "id": [self.id],
                            "evento": ["ESTACIONO"],
                            "time": [self.env.now]
                        })] )
                        break
            #Si el carro se estaciono, este evento dicta cuando va a liberar el parqueo luego de X tiempo
            if piso_asignado:
                self.env.process(self.estacionado(piso_asignado))
            #El carro no tiene un piso asignado (todos los pisos estaban llenos)
            else:
                print(f"{self.env.now}: El carro {self.id} se fue sin estacionarse. No habia espacio en ningun piso")
                self.eventos = pd.concat( [self.eventos, pd.DataFrame({
                    "id": [self.id],
                    "evento": ["NO ESTACIONO Y SALIO"],
                    "time": [self.env.now]
                })] )

    def estacionado(self, piso_asignado):
        tiempo_estacionado = random.randint(15,40)
        yield self.env.timeout(tiempo_estacionado)
        print(f"{self.env.now}: El carro {self.id} salio del piso {piso_asignado.id} despues de {tiempo_estacionado} minutos")
        # Hace lo contrario al get. Libera un espacio del contenedor
        yield piso_asignado.num_parqueos.put(1)
        self.eventos = pd.concat( [self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": ["SALIO"],
            "time": [self.env.now]
        })] )


La clase Carro tiene 2 funciones principales. 

La primer funcion start(), realiza la accion principal que es esperar en la cola de la entrada, entrar al estacionamiento y buscar un espacio disponible entre los distintos pisos del edificio de estacionamiento. La ultima parte de esta funcion ejecuta paralelamente la otra funcion estacionado().

La funcion estacionado() recibe el piso donde el carro se estaciono (recibido desde la funcion start()), esta funcion solamente va a esperar un tiempo aleatorio y luego de eso va a liberar un espacio del contenedor.


In [90]:
class Simulacion:
    def __init__(self, env, pisos_settings, num_parquimetros):
        self.env = env
        self.pisos = [Piso(env,p["id"], p["num_parqueos"]) for p in pisos_settings]
        self.estacionamiento = Estacionamiento(env, num_parquimetros, self.pisos)
        self.carros = []
        self.env.process(self.start())

    def start(self):
        while True:
            #Llegada de los carros a la entrada, entre 1 y 2 minutos.
            yield self.env.timeout(random.randint(1,2))
            #Puede llegar de 1 a 3 carros al mismo tiempo
            for _ in range(random.randint(1,3)):
                carro = Carro(
                    self.env,
                    len(self.carros)+1,
                    random.choice(self.pisos),
                    self.estacionamiento
                )
                self.carros.append(carro)

### Inicio de Simulacion

In [91]:
env = simpy.Environment()

pisos_settings = [
    {
        "id":1,
        "num_parqueos":15
    },
    {
        "id":2,
        "num_parqueos":15
    }
    ]

num_parquimetros = 2

simulacion = Simulacion(env,pisos_settings,num_parquimetros)

env.run(until=250)

1: el carro con id 1 llego y quiere estacionarse en el piso 1
1: el carro con id 2 llego y quiere estacionarse en el piso 1
1: el carro con id 3 llego y quiere estacionarse en el piso 2
2: el carro con id 4 llego y quiere estacionarse en el piso 1
2: el carro con id 5 llego y quiere estacionarse en el piso 1
2: El carro con id 1 esta entrando al estacionamiento
2: El carro con id 2 esta entrando al estacionamiento
2: El carro 1 se estaciono en el piso que queria (1)
2: El carro 2 se estaciono en el piso que queria (1)
2: El carro con id 3 esta entrando al estacionamiento
2: El carro 3 se estaciono en el piso que queria (2)
2: El carro con id 5 esta entrando al estacionamiento
2: El carro 5 se estaciono en el piso que queria (1)
3: el carro con id 6 llego y quiere estacionarse en el piso 2
3: El carro con id 4 esta entrando al estacionamiento
3: El carro 4 se estaciono en el piso que queria (1)
3: El carro con id 6 esta entrando al estacionamiento
3: El carro 6 se estaciono en el piso q

In [92]:
for s in simulacion.estacionamiento.pisos:
    print(f"{s.id}: {s.num_parqueos.level}")

1: 1
2: 1


### Construcción del DataFrame del perfil de los autos

In [93]:
lista_autos = []
for c in simulacion.carros:
    lista_autos.append({
        "id": c.id,
        "piso": c.piso.id,
        "encontro_parqueo": c.encontro_parqueo
    })

In [94]:
df_autos = pd.DataFrame(lista_autos)

### Construcción del DataFrame de eventos

In [106]:
simulacion.carros[35].eventos

,id,evento,time
0,36,LLEGO,30
0,36,ENTRO,31
0,36,ESTACIONO,31
0,36,SALIO,71
